In [1]:
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import statsmodels.stats.multitest as smm
from scipy import stats
import seaborn as sns

from Bio import Seq

pd.options.mode.chained_assignment = None

#makes viewing pandas tables better
pd.set_option('display.max_colwidth', 0)

Change the following parameters to your output directory (where you want output files to be written), your metafile, and TNseq output. 

If these files aren't in the same directory as this notebook, you need use the full path. For example, the full path of 'barseq_output' would be '/usr2/people/clairedubin/barseq/barseq_output'

Also change the control and experimental temperatures if needed.

In [2]:
#where the BarSeq FASTQs are
FASTQ_directory = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024'


## Barseq metafile

You'll need to create your own metafile - easiest way is edit in Excel. A sample metafile is located at: /usr2/people/clairedubin/barseq/Kluyv_BarSeq_metadata.txt

    Fastq: path to barseq FASTQ for each competition
    SampleName: unique name for each competition, MUST include ctrl_temp or exp_temp as specified above
    DualIndex: index from sequencing (in read name lines of FASTQ and the sequencing info file that Adam sends)
    
    UsePrecounted: True if you've already run the Barseq counting script on a competition, False otherwise
    
    poolfile: path to Tnseq output file, should be the same for each competition
    output_dir: path to output directory
    minRandom: number of bases between start of read and DualIndex, usually in sequence of 1,2,3,4,1,2,3,4 for each sample
    maxRandom: same as minRandom
    
    BeforeBarcode: sequence preceding all barcodes
    AfterBarcode: sequence following all barcodes
    BarcodeLengths: lengths of barcodes to search for, with expected length of barcode first

    

In [3]:
!ls {FASTQ_directory}/*fastq

/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/37A1_S1_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/37A2_S2_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/37B1_S3_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/37B2_S4_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/50A1_S5_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/50A2_S6_L001

In [4]:
#format: sample_name, fastq_path, dual_index, offset
#add or delete more lines as needed

sample_info = [('4k_37_A1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/OORB010_S1_L001_R1_001.fastq', 'ATCACG', '1'),
               ('4k_37_A2', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/OORB011_S1_L001_R1_001.fastq', 'CGATGT', '2'), 
               ('4k_37_B1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/KFRB001_S1_L001_R1_001.fastq', 'ATCACG', '1'),
               ('4k_37_B2', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/KFRB002_S1_L001_R1_001.fastq', 'CGATGT', '2'),
               ('4k_37_C1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/KFRB011_S1_L001_R1_001.fastq', 'ATCACG', '1'),
               ('4k_37_C2', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/KFRB012_S1_L001_R1_001.fastq', 'CGATGT', '2'), 
               ('4k_50_A1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/KFRB005_S1_L001_R1_001.fastq', 'ATCACG', '1'),
               ('4k_50_A2', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/KFRB006_S1_L001_R1_001.fastq', 'CGATGT', '2'),
               ('4k_50_B1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/KFRB013_S1_L001_R1_001.fastq', 'ATCACG', '1'),
               ('4k_50_B2', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/KFRB014_S1_L001_R1_001.fastq', 'CGATGT', '2'),
               ('13k_37_A1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/37A1_S1_L001_R1_001.fastq', 'ATCACG', '1'),
               ('13k_37_A2', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/37A2_S2_L001_R1_001.fastq', 'TTAGGC', '3'),
               ('13k_37_B1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/37B1_S3_L001_R1_001.fastq', 'CGATGT', '2'),
               ('13k_37_B2', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/37B2_S4_L001_R1_001.fastq', 'TGACCA', '4'),
               ('13k_50_A1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/50A1_S5_L001_R1_001.fastq', 'ACAGTG', '1'),
               ('13k_50_A2', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/50A2_S6_L001_R1_001.fastq', 'CAGATC', '3'),
               ('13k_50_B1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/50B1_S7_L001_R1_001.fastq', 'GCCAAT', '2'),
               ('13k_50_B2', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/50B2_S8_L001_R1_001.fastq', 'ACTTGA', '4'),
                ]

In [5]:
!head /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/OORB010_S1_L001_R1_001.fastq

@M07716:180:000000000-DMYDD:1:1101:15574:1331 1:N:0:ATCACG
AGCACTAGTCGACCTGCAGCGTACGGGTGGTGAATACAGGTCATCAGAGACCTCGTGGACATCAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATCACGATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAA
+
CDDDDFFFFFDDGGGGGGGGGGGGGGGGGGGGHHHHHHHHHHHHHHHHHHHGHHGHHGGHHHHHHHHHHGGGGHHHHHHHGHHHGHHHHHHHHHHHHHHHHHGFHFHHHGGHHHHGGGHGHHHHHHHHHHHHGGGGF-<-
@M07716:180:000000000-DMYDD:1:1101:15691:1331 1:N:0:ATCACG
TGCACTAGTCGACCTGCAGCGTACGTCATCATGGACCCGTGTAATAGAGACCTCGTGGACATCAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATCACGATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAA
+
DCDDDFFFFFCDGGGGGGGGGGGGGHF225DA53BA2EE2AAAFGHHHHHHGHHGHHGGHHHHHHHHHHGGGGHHHHHHHGFF2?@F3B3F3@BBB334B3?/>/?BF0?/33?F<?<BF3?B3?F20FGHHGGGGG<<<
@M07716:180:000000000-DMYDD:1:1101:15669:1331 1:N:0:ATCACG
AGCACTAGTCGACCTGCAGCGTACGACGGTGTTCCAATTGTGGGCAGAGACCTCGTGGACATCAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATCACGATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAA


### To convert the annotated file to csv and not to be separated by tab (execute the next three lines only if necessary)

In [7]:
poolfile1 = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_mapping_output_4k_13k_mutants_all_fastqs_available/pool_4k_plus_13k_combined_poolfile_with_promoter_reannotated.csv'
df=pd.read_csv(poolfile1,sep='\t')

In [8]:
df.to_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_mapping_output_4k_13k_mutants_all_fastqs_available/pool_4k_plus_13k_combined_poolfile_with_promoter_reannotated_1.csv',index=None)

In [9]:
df=pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_mapping_output_4k_13k_mutants_all_fastqs_available/pool_4k_plus_13k_combined_poolfile_with_promoter_reannotated_1.csv')

In [10]:
#these variables should be the same for each replicate

metafile_name = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/BarSeq_metafile_13k_mutants_37•C_vs_50•C_12_19_2024.txt'
logfile_name = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/BarSeq_metafile_13k_mutants_37•C_vs_50•C_12_19_2024.log'
poolfile = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_mapping_output_4k_13k_mutants_all_fastqs_available/pool_4k_plus_13k_combined_poolfile_with_promoter_reannotated_1.csv'
output_dir = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/Barseq_recounted_with_promoter'
before_barcode = 'GTCGACCTGCAGCGTACG' #bottom strand sequence was used
#before_barcode = 'GATGTCCACGAGGTCTCT' #what was there before 
after_barcode =  'AGAGACCTCGTGGACATC' #bottom strand sequence was used
#after_barcode = 'CGTACGCTGCAGGTCGAC'  #what was there before 
barcode_lengths = '20,19,18,21'

In [11]:
#if there are any samples where you already counted barcodes, add the sample names here
precounted_samples = []

In [12]:
metafile_columns = ['FileIndex','SampleName', 'Fastq', 'DualIndex', 'Poolfile', 'OutputDir', 'minRandom', 'maxRandom',
                   'BeforeBarcode', 'AfterBarcode', 'BarcodeLengths', 'UsePrecounted']

with open(metafile_name, 'w') as f:
    
    #write column names
    f.write('\t'.join(metafile_columns)+'\n')
    
    #write a line for each pool
    for sample_name, fastq, dual_index, offset in sample_info:
        
        #DUAL INDECES ARE BACKWARDS FROM WHAT THE FLOWCELL OUTPUT SAYS
        to_write = '\t'.join([sample_name, sample_name, fastq, dual_index[::-1], poolfile, output_dir, offset, offset,
                             before_barcode, after_barcode, barcode_lengths, 'False'])
        f.write(to_write+'\n')
        
    for precounted_sample_name in precounted_samples:
        
        to_write = '\t'.join([sample_name, sample_name, fastq, dual_index, poolfile, output_dir, offset, offset,
                             before_barcode, after_barcode, barcode_lengths, 'True'])




### Check metafile

In [13]:
pd.read_csv(metafile_name, sep='\t')

,FileIndex,SampleName,Fastq,DualIndex,Poolfile,OutputDir,minRandom,maxRandom,BeforeBarcode,AfterBarcode,BarcodeLengths,UsePrecounted
0,4k_37_A1,4k_37_A1,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/OORB010_S1_L001_R1_001.fastq,GCACTA,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_mapping_output_4k_13k_mutants_all_fastqs_available/pool_4k_plus_13k_combined_poolfile_with_promoter_reannotated_1.csv,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/Barseq_recounted_with_promoter,1,1,GTCGACCTGCAGCGTACG,AGAGACCTCGTGGACATC,"20,19,18,21",False
1,4k_37_A2,4k_37_A2,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/OORB011_S1_L001_R1_001.fastq,TGTAGC,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_mapping_output_4k_13k_mutants_all_fastqs_available/pool_4k_plus_13k_combined_poolfile_with_promoter_reannotated_1.csv,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/Barseq_recounted_with_promoter,2,2,GTCGACCTGCAGCGTACG,AGAGACCTCGTGGACATC,"20,19,18,21",False
2,4k_37_B1,4k_37_B1,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/KFRB001_S1_L001_R1_001.fastq,GCACTA,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_mapping_output_4k_13k_mutants_all_fastqs_available/pool_4k_plus_13k_combined_poolfile_with_promoter_reannotated_1.csv,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/Barseq_recounted_with_promoter,1,1,GTCGACCTGCAGCGTACG,AGAGACCTCGTGGACATC,"20,19,18,21",False
3,4k_37_B2,4k_37_B2,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/KFRB002_S1_L001_R1_001.fastq,TGTAGC,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_mapping_output_4k_13k_mutants_all_fastqs_available/pool_4k_plus_13k_combined_poolfile_with_promoter_reannotated_1.csv,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/Barseq_recounted_with_promoter,2,2,GTCGACCTGCAGCGTACG,AGAGACCTCGTGGACATC,"20,19,18,21",False
4,4k_37_C1,4k_37_C1,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/BarSeq_FASTQs_4k_plus_13k_mutants_11_04_2024/KFRB011_S1_L001_R1_001.fastq,GCACTA,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_mapping_output_4k_13k_mutants_all_fastqs_available/pool_4k_plus_13k_combined_poolfile_with_promoter_reannotated_1.csv,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/Barseq_recounted_with_promoter,1,1,GTCGACCTGCAGCGTACG,AGAGACCTCGTGGACATC,"20,19,18,21",False
5,

## Barseq counting

Might take a few hours!

In [14]:
!python3 /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/RBseq_Count_BarCodes_v2.py -m {metafile_name} -l {logfile_name}

2024-12-19 14:09:07 RBseq_Count_BarCodes.py
2024-12-19 14:09:07 Version: 1.1.4
2024-12-19 14:09:07 Release Date: July 16, 2020
2024-12-19 14:09:07 Options passed:  metafile:/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/BarSeq_metafile_13k_mutants_37•C_vs_50•C_12_19_2024.txt  logFile:/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/BarSeq_metafile_13k_mutants_37•C_vs_50•C_12_19_2024.log  minQual:10  matchBefore:6  matchAfter:6  quietMode:False 
2024-12-19 14:09:07 Logging status updates in /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/BarSeq_metafile_13k_mutants_37•C_vs_50•C_12_19_2024.log
2024-12-19 14:09:07 Loading TnSeq library metadata from /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_

2024-12-19 14:09:52     1279289 reads processed.
2024-12-19 14:09:52     1228497 reads with compliant barcodes.
2024-12-19 14:09:52     50792 reads without recognizable, compliant barcodes. Of those:
2024-12-19 14:09:52       20511 reads without expected Dual Index.
2024-12-19 14:09:52       23903 reads without expected sequence before the barcode region.
2024-12-19 14:09:52       6378 reads without expeced sequence after the barcode region.
2024-12-19 14:09:52       0 reads with noncompliant barcdes. (Contains Ns, etc).
2024-12-19 14:09:52       0 reads with quality scores less than 10
2024-12-19 14:09:52   Saving barcode counts to /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/Barseq_recounted_with_promoter/countsFiles/4k_37_B2.counts
2024-12-19 14:09:52   Total barcodes seen (incudes sequencing errors): 57482

2024-12-19 14:10:30     1115874 reads processed.
2024-12-19 14:10:30     1078435 reads with compliant barcodes.
2024-12-19 14:10:30     37439 reads without recognizable, compliant barcodes. Of those:
2024-12-19 14:10:30       16098 reads without expected Dual Index.
2024-12-19 14:10:30       16523 reads without expected sequence before the barcode region.
2024-12-19 14:10:30       4818 reads without expeced sequence after the barcode region.
2024-12-19 14:10:30       0 reads with noncompliant barcdes. (Contains Ns, etc).
2024-12-19 14:10:30       0 reads with quality scores less than 10
2024-12-19 14:10:30   Saving barcode counts to /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/Barseq_recounted_with_promoter/countsFiles/4k_50_A2.counts
2024-12-19 14:10:30   Total barcodes seen (incudes sequencing errors): 57745

2024-12-19 14:16:12 ---------------------
2024-12-19 14:16:12   Mapping reads from /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/37A2_S2_L001_R1_001.fastq
2024-12-19 14:16:12   Filtering reads on second index: CGGATT
2024-12-19 14:16:12   Looking for sequence barcode preceeded by CGTACG
2024-12-19 14:21:21     39219999 reads processed.
2024-12-19 14:21:21     36060633 reads with compliant barcodes.
2024-12-19 14:21:21     3159366 reads without recognizable, compliant barcodes. Of those:
2024-12-19 14:21:21       2394780 reads without expected Dual Index.
2024-12-19 14:21:21       569716 reads without expected sequence before the barcode region.
2024-12-19 14:21:21       194870 reads without expeced sequence after the barcode region.
2024-12-19 14:21:21       0 reads with noncompliant barcdes. (Contains Ns, etc).
2024-12-19 14:21:21       0 reads with quality scores les

2024-12-19 14:38:24   Mapping reads from /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/BarSeq_FASTQs_13k_mutants_03_27_2024/Barseq_37•C_vs_50•C_500_million_reads/50A2_S6_L001_R1_001.fastq
2024-12-19 14:38:24   Filtering reads on second index: CTAGAC
2024-12-19 14:38:24   Looking for sequence barcode preceeded by CGTACG
2024-12-19 14:42:53     38141465 reads processed.
2024-12-19 14:42:53     35591327 reads with compliant barcodes.
2024-12-19 14:42:53     2550138 reads without recognizable, compliant barcodes. Of those:
2024-12-19 14:42:53       1637925 reads without expected Dual Index.
2024-12-19 14:42:53       710264 reads without expected sequence before the barcode region.
2024-12-19 14:42:53       201949 reads without expeced sequence after the barcode region.
2024-12-19 14:42:53       0 reads with noncompliant barcdes. (Contains Ns, etc).
2024-12-19 14:42:53       0 reads with quality scores less than 10
2024-12-19 14:42:53   Saving bar

#### To change the Barseq pool count file from txt to csv for making plots

In [15]:
import pandas as pd

# Load the data from the TSV file
tsv_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_12_19_2024/Barseq_recounted_with_promoter/poolCount.txt'
df = pd.read_csv(tsv_file_path, sep='\t')

# Save the data to a CSV file
csv_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_12_19_2024/Barseq_recounted_with_promoter/poolCount.csv'
df.to_csv(csv_file_path, index=False)

# Making Scatter Plots

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/poolCount.txt', sep='\t')  # Update with your file path

# Define the pairs of columns for scatter plots
column_pairs = [
    ('4k_37_A1', '4k_37_A2'),
    ('4k_37_B1', '4k_37_B2'),
    ('4k_37_C1', '4k_37_C2'),
    ('4k_50_A1', '4k_50_A2'),
    ('4k_50_B1', '4k_50_B2'),
    ('13k_37_A1', '13k_37_B2'),
    ('13k_37_B1', '13k_37_B2'),
    ('13k_50_A1', '13k_50_B2'),
    ('13k_50_B1', '13k_50_B2')
]

# Plotting each pair of columns
for x_column, y_column in column_pairs:
    plt.figure(figsize=(8, 6))
    plt.scatter(df[x_column], df[y_column], color='black')
    plt.xlabel(f'{x_column} Abundance', fontsize=11)
    plt.ylabel(f'{y_column} Abundance', fontsize=11)
    plt.title(f'Scatter Plot: {x_column} vs {y_column}', fontsize=13, color='purple')
    plt.show()


### Ajusting the axis of the scatter plots

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/poolCount.txt', sep='\t')  # Update with your file path

# Define the pairs of columns for scatter plots
column_pairs = [
    ('4k_37_A1', '4k_37_A2'),
    ('4k_37_B1', '4k_37_B2'),
    ('4k_37_C1', '4k_37_C2'),
    ('4k_50_A1', '4k_50_A2'),
    ('4k_50_B1', '4k_50_B2'),
    ('13k_37_A1', '13k_37_A2'),
    ('13k_37_B1', '13k_37_B2'),
    ('13k_50_A1', '13k_50_A2'),
    ('13k_50_B1', '13k_50_B2')
]


# Set custom axis limits for each pair (adjust as needed)
axis_limits = {
    '4k_37_A1_4k_37_A2': {'xlim': (0, 20000), 'ylim': (0, 20000)},
    '4k_37_B1_4k_37_B2': {'xlim': (0, 10000), 'ylim': (0, 10000)},
    '4k_37_C1_4k_37_C2': {'xlim': (0, 10000), 'ylim': (0, 10000)},
    '4k_50_A1_4k_50_A2': {'xlim': (0, 6000), 'ylim': (0, 6000)},
    '4k_50_B1_4k_50_B2': {'xlim': (0, 6000), 'ylim': (0, 6000)},
    '13k_37_A1_13k_37_A2': {'xlim': (0, 250000), 'ylim': (0, 250000)},
    '13k_37_B1_13k_37_B2': {'xlim': (0, 250000), 'ylim': (0, 250000)},
    '13k_50_A1_13k_50_A2': {'xlim': (0, 250000), 'ylim': (0, 250000)},
    '13k_50_B1_13k_50_B2': {'xlim': (0, 250000), 'ylim': (0, 250000)},
}

# Plotting each pair of columns with custom axis limits
for x_column, y_column in column_pairs:
    key = f'{x_column}_{y_column}'
    plt.figure(figsize=(8, 6))
    plt.scatter(df[x_column], df[y_column], color='black')
    plt.xlabel(f'{x_column} Abundance', fontsize=11)
    plt.ylabel(f'{y_column} Abundance', fontsize=11)
    plt.title(f'Scatter Plot: {x_column} vs {y_column}', fontsize=13, color='purple')
    
    # Apply axis limits if available
    if key in axis_limits:
        plt.xlim(axis_limits[key]['xlim'])
        plt.ylim(axis_limits[key]['ylim'])

    plt.show()


## to normalize the Barseq pool count file based on the number of reads obtained

### this is the normalization that worked with the right result

In [ ]:
import pandas as pd

# Load the count file into a DataFrame
count_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/poolCount.txt'
df = pd.read_csv(count_file_path, sep='\t')

# Calculate the total counts for each replicate
df['Total_Count_4k_37_A1'] = df['4k_37_A1'].sum()
df['Total_Count_4k_37_A2'] = df['4k_37_A2'].sum()
df['Total_Count_4k_37_B1'] = df['4k_37_B1'].sum()
df['Total_Count_4k_37_B2'] = df['4k_37_B2'].sum()
df['Total_Count_4k_37_C1'] = df['4k_37_C1'].sum()
df['Total_Count_4k_37_C2'] = df['4k_37_C2'].sum()
df['Total_Count_4k_50_A1'] = df['4k_50_A1'].sum()
df['Total_Count_4k_50_A2'] = df['4k_50_A2'].sum()
df['Total_Count_4k_50_B1'] = df['4k_50_B1'].sum()
df['Total_Count_4k_50_B2'] = df['4k_50_B2'].sum()
df['Total_Count_13k_37_A1'] = df['13k_37_A1'].sum()
df['Total_Count_13k_37_A2'] = df['13k_37_A2'].sum()
df['Total_Count_13k_37_B1'] = df['13k_37_B1'].sum()
df['Total_Count_13k_37_B2'] = df['13k_37_B2'].sum()
df['Total_Count_13k_50_A1'] = df['13k_50_A1'].sum()
df['Total_Count_13k_50_A2'] = df['13k_50_A2'].sum()
df['Total_Count_13k_50_B1'] = df['13k_50_B1'].sum()
df['Total_Count_13k_50_B2'] = df['13k_50_B2'].sum()



# Normalize counts for each replicate by dividing by its own Total_Count
df['normalized_4k_37_A1'] = df['4k_37_A1'] / df['Total_Count_4k_37_A1']
df['normalized_4k_37_A2'] = df['4k_37_A2'] / df['Total_Count_4k_37_A2']
df['normalized_4k_37_B1'] = df['4k_37_B1'] / df['Total_Count_4k_37_B1']
df['normalized_4k_37_B2'] = df['4k_37_B2'] / df['Total_Count_4k_37_B2']
df['normalized_4k_37_C1'] = df['4k_37_C1'] / df['Total_Count_4k_37_C1']
df['normalized_4k_37_C2'] = df['4k_37_C2'] / df['Total_Count_4k_37_C2']
df['normalized_4k_50_A1'] = df['4k_50_A1'] / df['Total_Count_4k_50_A1']
df['normalized_4k_50_A2'] = df['4k_50_A2'] / df['Total_Count_4k_50_A2']
df['normalized_4k_50_B1'] = df['4k_50_B1'] / df['Total_Count_4k_50_B1']
df['normalized_4k_50_B2'] = df['4k_50_B2'] / df['Total_Count_4k_50_B2']
df['normalized_13k_37_A1'] = df['13k_37_A1'] / df['Total_Count_13k_37_A1']
df['normalized_13k_37_A2'] = df['13k_37_A2'] / df['Total_Count_13k_37_A2']
df['normalized_13k_37_B1'] = df['13k_37_B1'] / df['Total_Count_13k_37_B1']
df['normalized_13k_37_B2'] = df['13k_37_B2'] / df['Total_Count_13k_37_B2']
df['normalized_13k_50_A1'] = df['13k_50_A1'] / df['Total_Count_13k_50_A1']
df['normalized_13k_50_A2'] = df['13k_50_A2'] / df['Total_Count_13k_50_A2']
df['normalized_13k_50_B1'] = df['13k_50_B1'] / df['Total_Count_13k_50_B1']
df['normalized_13k_50_B2'] = df['13k_50_B2'] / df['Total_Count_13k_50_B2']

# Save the normalized count file
normalized_count_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/poolCount_4k_plus_13k_mutants_37•C_vs_50•C.csv'
df.to_csv(normalized_count_file_path, index=False)


##### To make the fitness abundance agaian using the normalized values

In [ ]:
import pandas as pd

# Load the CSV file
csv_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/poolCount_4k_plus_13k_mutants_37•C_vs_50•C.csv'
df = pd.read_csv(csv_file_path)

# Save the DataFrame to a text file (txt)
txt_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/poolCount_4k_plus_13k_mutants_37•C_vs_50•C.txt'
df.to_csv(txt_file_path, sep='\t', index=False)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_13k_mutants_37•C_vs_50•C_10_28_2024/poolCount_37•C_vs_50•C.txt', sep='\t')

# Define the pairs of columns for scatter plots
column_pairs = [
    ('normalized_37_A1', 'normalized_37_A2'),
    ('normalized_37_B1', 'normalized_37_B2'),
    ('normalized_50_A1', 'normalized_50_A2'),
    ('normalized_50_B1', 'normalized_50_B2')
]

# Plotting each pair of columns
for x_column, y_column in column_pairs:
    plt.figure(figsize=(8, 6))
    plt.scatter(df[x_column], df[y_column], color='black')
    plt.xlabel(f'{x_column} Abundance', fontsize=11)
    plt.ylabel(f'{y_column} Abundance', fontsize=11)
    plt.title(f'Scatter Plot: {x_column} vs {y_column}', fontsize=13, color='purple')
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_13k_mutants_37•C_vs_50•C_10_28_2024/poolCount_37•C_vs_50•C.txt', sep='\t')

# Define the pairs of columns and their axis limits for scatter plots
column_pairs = [
    ('normalized_37_A1', 'normalized_37_A2'),
    ('normalized_37_B1', 'normalized_37_B2'),
    ('normalized_50_A1', 'normalized_50_A2'),
    ('normalized_50_B1', 'normalized_50_B2')
]

# Set custom axis limits for each pair (adjust as needed)
axis_limits = {
    'normalized_37_A1_normalized_37_A2': {'xlim': (0, 0.02), 'ylim': (0, 0.025)},
    'normalized_37_B1_normalized_37_B2': {'xlim': (0, 0.02), 'ylim': (0, 0.025)},
    'normalized_50_A1_normalized_50_A2': {'xlim': (0, 0.02), 'ylim': (0, 0.02)},
    'normalized_50_B1_normalized_50_B2': {'xlim': (0, 0.02), 'ylim': (0, 0.02)}
}

# Plotting each pair of columns with custom axis limits
for x_column, y_column in column_pairs:
    key = f'{x_column}_{y_column}'
    plt.figure(figsize=(8, 6))
    plt.scatter(df[x_column], df[y_column], color='black')
    plt.xlabel(f'{x_column} Abundance', fontsize=11)
    plt.ylabel(f'{y_column} Abundance', fontsize=11)
    plt.title(f'Scatter Plot: {x_column} vs {y_column}', fontsize=13, color='purple')
    
    # Apply axis limits if available
    if key in axis_limits:
        plt.xlim(axis_limits[key]['xlim'])
        plt.ylim(axis_limits[key]['ylim'])

    plt.show()


# To do statistical t-test

### To handle rows where some values are zero and convert them to NaN before performing the Mann-Whitney U test

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu

# Load the dataset
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/poolCount_4k_plus_13k_mutants_37•C_vs_50•C.csv')

# Convert all columns to numeric, coercing errors to NaN
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Replace zeros with NaN
df.replace(0, np.nan, inplace=True)

# Define the data columns for the existing and new conditions
data_37C = df[['normalized_4k_37_A1', 'normalized_4k_37_A2', 'normalized_4k_37_B1', 'normalized_4k_37_B2', 'normalized_4k_37_C1', 'normalized_4k_37_C2', 'normalized_13k_37_A1', 'normalized_13k_37_A2', 'normalized_13k_37_B1', 'normalized_13k_37_B2']]
data_50C = df[['normalized_4k_50_A1', 'normalized_4k_50_A2', 'normalized_4k_50_B1', 'normalized_4k_50_B2', 'normalized_13k_50_A1', 'normalized_13k_50_A2', 'normalized_13k_50_B1', 'normalized_13k_50_B2']]
            



# List to store p-values for existing conditions
p_values_37_vs_50 = []

# Iterate over rows and perform Mann-Whitney U test for each
for index, row in df.iterrows():
    # Drop NaN values from the datasets
    data_37C_values = row[data_37C.columns].dropna()
    
    

    # Perform the Mann-Whitney U test for 37C vs. 50C
    #data_37C_values = row[data_25C.columns].dropna()
    data_50C_values = row[data_50C.columns].dropna()

    if len(data_37C_values) > 0 and len(data_50C_values) > 0:
        statistic, p_value = mannwhitneyu(data_37C_values, data_50C_values, alternative='two-sided')
        p_values_37_vs_50.append(p_value)
    else:
        p_values_37_vs_50.append(np.nan)

# Add the p-values to the DataFrame

df['MannWhitneyU_p_value_37_vs_50'] = p_values_37_vs_50

# Save the DataFrame with the calculated p-values to a new CSV file
output_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_4k_plus_13k_mutants/for_analysis/pool_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_4k_plus_13k_mutants_37•C_vs_50•C_11_05_2024/poolCount_4k_plus_13k_mutants_37•C_vs_50•C_output_with_mannwhitneyu_p_values_zero_values_removed_combined.csv'
df.to_csv(output_path, index=False)


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu

# Load the dataset
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_13k_mutants_37•C_vs_50•C_10_28_2024/poolCount_37•C_vs_50•C.csv')

# Convert all columns to numeric, coercing errors to NaN
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Replace zeros with NaN
df.replace(0, np.nan, inplace=True)

# Define the data columns for the existing and new conditions
data_37C = df[['37_A1', '37_A2', '37_B1', '37_B2']]
data_50C = df[['50_A1', '50_A2', '50_B1', '50_B2']]
            



# List to store p-values for existing conditions
p_values_37_vs_50 = []

# Iterate over rows and perform Mann-Whitney U test for each
for index, row in df.iterrows():
    # Drop NaN values from the datasets
    data_37C_values = row[data_37C.columns].dropna()
    
    

    # Perform the Mann-Whitney U test for 37C vs. 50C
    #data_37C_values = row[data_25C.columns].dropna()
    data_50C_values = row[data_50C.columns].dropna()

    if len(data_37C_values) > 0 and len(data_50C_values) > 0:
        statistic, p_value = mannwhitneyu(data_37C_values, data_50C_values, alternative='two-sided')
        p_values_37_vs_50.append(p_value)
    else:
        p_values_37_vs_50.append(np.nan)

# Add the p-values to the DataFrame

df['MannWhitneyU_p_value_37_vs_50'] = p_values_37_vs_50

# Save the DataFrame with the calculated p-values to a new CSV file
output_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_barseq_outputs_using_all_fastqs_combined_13k_mutants_37•C_vs_50•C_10_28_2024/poolCount__raw_37•C_vs_50•C_output_with_mannwhitneyu_p_values_zero_values_removed_combined.csv'
df.to_csv(output_path, index=False)
